In [0]:
import tensorflow as tf 

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

In [0]:
def layer(input,weight_shape , bias_shape):
    weightdev=(2.0/weight_shape[0])**0.5
    winit=tf.random_normal_initializer(stddev=weightdev)
    binit=tf.constant_initializer(value=0)
    W=tf.get_variable(shape=weight_shape , initializer=winit,dtype='float32',name='weight')
    b=tf.get_variable(shape=bias_shape , initializer=binit ,dtype='float32',name='bias')
    output = tf.nn.relu(tf.matmul(input,W)+b)
    return output 

In [0]:
def conv2d(input , weight_shape , bias_shape):
    inn = weight_shape[0] * weight_shape[1] * weight_shape[2]
    weight_init = tf.random_normal_initializer(stddev=
                                              (2.0/inn)**0.5)
    bias_init=tf.constant_initializer(value=0)
    W=tf.get_variable(name='weight',shape=weight_shape,initializer=weight_init,dtype='float32')
    b=tf.get_variable(name='bias',shape=bias_shape,initializer=bias_init,dtype='float32')
    out=tf.nn.conv2d(input,filter=W,strides=[1,1,1,1],padding='SAME',name='conv')
    return tf.nn.relu(tf.nn.bias_add(out,b))

In [0]:
def maxpool(input,k=2):
    return tf.nn.max_pool(input,ksize=[1,k,k,1],strides=[1,k,k,1],padding='SAME')

In [0]:
def inference(X,keep_prob):
    x=tf.reshape(X, shape=[-1,28,28,1])
    with tf.variable_scope("conv1"):
        conv1=conv2d(x,[5,5,1,32],[32])
        pool1=maxpool(conv1)
    with tf.variable_scope("conv2"):
        conv2=conv2d(pool1,[5,5,32,64],[64])
        pool2=maxpool(conv2)
    with tf.variable_scope("fc"):
        pool1_flat=tf.reshape(pool2,shape=[-1,7*7*64])
        fc_1=layer(pool1_flat,[7*7*64,1024],[1024])
        fc_drop=tf.nn.dropout(fc_1,keep_prob)
    with tf.variable_scope("output"):
        output=layer(fc_drop,[1024,10],[10])
    return output

In [0]:
x=tf.placeholder(dtype='float32',shape=[None, 784 ])
y=tf.placeholder(dtype='float32',shape=[None , 10])

In [0]:
y_=inference(x,keep_prob=0.5)

In [9]:
crossentropy=tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=y_)
loss=tf.reduce_mean(crossentropy)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [0]:
train_op=tf.train.AdamOptimizer(1e-4).minimize(loss)


In [19]:
sess=tf.InteractiveSession()
tf.global_variables_initializer().run()

/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1714: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [0]:
for i in range(20000):
    batch_xs , batch_ys =mnist.train.next_batch(100)
    sess.run(train_op ,feed_dict={x : batch_xs , y : batch_ys})

In [0]:
correct_pred=tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
accuracy=tf.reduce_mean(tf.cast(correct_pred , tf.float32))

In [22]:
print (sess.run(accuracy,feed_dict={x:mnist.test.images , y : mnist.test.labels}))

0.9925
